# Messaging


### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
import asyncio


from libs.messaging_service import MessagingService

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://demo-participant-agent:3021 and an api key of demo-participantsSecretApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Instatiate Messaging Service

In [4]:
messaging_service = MessagingService(agent_controller)

## Add Contact

### Either from Invitation Shared

In [ ]:
invitation = {"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "a6620684-965c-4f35-90e8-35e6093af9f0", "label": "Hyperledger Global Forum", "recipientKeys": ["B6daDADbT7RUUB3Qkap4QnJDEYVhXjHnAboRevBQCCC1"], "serviceEndpoint": "https://3b29be66da95.ngrok.io"}


In [ ]:
new_contact_id = messaging_service.accept_contact_invitation(invitation, "HLGF")

### Or by Creating and Sharing an Invite

In [6]:
alias = "HLGF"
response = messaging_service.new_contact_invite(alias)

# If you want to display as Q.R for example
invitation = response["invitation"]

new_contact_id = response["connection_id"]

Contact with ID f0f0f9e0-0ac3-4798-aa20-54fed69e98bc and alias HLGF added
----------------------------------------------------------------------------------------------------

Share this invite object with another entity you wish to add as a contact under alias HLGF

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "e751243f-9db3-4db8-bd66-ca252fda8289", "label": "Demo Participant", "recipientKeys": ["9vBrodjt67R2psV88YjPVXPxdTF3v8PFjdXHoLVUjg8x"], "serviceEndpoint": "https://0f8e21a7a624.ngrok.io"}


----------------------------------------------------------------------------------------------------
Contact with ID: f0f0f9e0-0ac3-4798-aa20-54fed69e98bc and alias HLGF is now active.
Message received from connection f0f0f9e0-0ac3-4798-aa20-54fed69e98bc with alias HLGF


## Send Message to Contact by Connection ID

In [7]:
some_arbitrary_sized_message = "Some second super super super great test message"
messaging_service.send_message(new_contact_id, some_arbitrary_sized_message)

## Display Inbox for Contact

In [8]:
messaging_service.display_inbox_for_contact(new_contact_id)

--------------------------------------------------
Inbox for Contact HLGF 

--------------------------------------------------
Connection ID : f0f0f9e0-0ac3-4798-aa20-54fed69e98bc 
Agent Label : Hyperledger Global Forum 
Total Messages : 2

--------------------------------------------------
Wed Jun  2 13:39:51 2021
Hello 
--------------------------------------------------
Wed Jun  2 13:39:55 2021
                         Some second super super super 
                         great test message 
--------------------------------------------------


## Get Contacts

In [9]:
messaging_service.view_inbox()

--------------------------------------------------
1 Contacts
--------------------------------------------------
Contact HLGF with ID f0f0f9e0-0ac3-4798-aa20-54fed69e98bc 
Status : ACTIVEMessages 2 
--------------------------------------------------



In [ ]:
response = messaging_service.get_contacts()
for contact in response:
    print(f"{contact.alias} : {contact.connection_id}")

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()